<a href="https://colab.research.google.com/github/elio-li/CSCI4964/blob/main/hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework 2**

In [ ]:
# Dataset Link: https://archive-beta.ics.uci.edu/ml/datasets/spambase
# Some Pre-Setup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Read File, Get x & y
df = pd.read_csv("admission.csv")
df = df.rename(columns={'Serial No.': 'id', 'GRE Score': 'gre', 'TOEFL Score': 'toefl',
                        'University Rating': 'rating', 'SOP': 'sop', 'LOR ': 'lor',
                        'CGPA': 'cgpa', 'Research': 'research', 'Chance of Admit ': 'admit'})
y = np.where(df['admit'] >= df['admit'].median(), 1, 0)
x = df.drop(columns={'admit', 'id'}, axis = 1)

# Split Data into Training Sets & Test Sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

## **Task 1 (30 points)**:

Implement a Decision Tree Classifier for your classification problem. You may use a built-in package to implement your classifier. Try modifying one or more of the input parameters and describe what changes you notice in your results. Clearly describe how these factors are affecting your output.

In [ ]:
from sklearn import tree
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score

# max_depth(None):       Any Int
# min_samples_split(2):  Any Int or Float
# max_features(None):    Int of 1~7
# min_impurity_split(0): Any Float
clf = tree.DecisionTreeClassifier(max_depth=5, min_samples_split=50, max_features=3, min_impurity_split=0.05)
clf = clf.fit(x_train, y_train)

# Get Prediction Result
y_pred = clf.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy: {:.2f}\nPrecision: {:.5f}\nRecall: {:.5f}\nF1: {:.5f}".format(accuracy, precision, recall, f1))

Accuracy: 0.90
Precision: 0.87755
Recall: 0.91489
F1: 0.89583


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/tree/_classes.py:316: FutureWarning: The min_impurity_split parameter is deprecated. Its default value has changed from 1e-7 to 0 in version 0.23, and it will be removed in 1.0 (renaming of 0.25). Use the min_impurity_decrease parameter instead.
  warnings.warn(


- Table of Accuracy, Precision, Recall & F1 Using Different Parameters

| # | MaxDepth | MinSmapleSplit | MaxFeatures | MinImpuritySplit | Accuracy | Precision | Recall | F1 |
| -- | ---- | --- | ---- | ---- | ---- | ------- | ------- | ------- |
| 1  | None | 2   | None | 0    | 0.84 | 0.80392 | 0.87234 | 0.83673 |
| 2  | 3    | 2   | 1    | 0    | 0.78 | 0.71186 | 0.89362 | 0.79245 |
| 3  | 5    | 2   | 2    | 0    | 0.89 | 0.87500 | 0.89362 | 0.88421 |
| 4  | 7    | 2   | 4    | 0    | 0.85 | 0.84783 | 0.82979 | 0.83871 |
| 5  | 99   | 2   | 7    | 0    | 0.84 | 0.81633 | 0.85106 | 0.83333 |
| 6  | 5    | 100 | 3    | 0    | 0.87 | 0.88636 | 0.82979 | 0.85714 |
| 7  | 5    | 70  | 3    | 0    | 0.89 | 0.84615 | 0.93617 | 0.88889 |
| 8  | 5    | 50  | 3    | 0    | 0.90 | 0.87755 | 0.91489 | 0.89583 |
| 9  | 99   | 50  | 7    | 0    | 0.92 | 0.91489 | 0.91489 | 0.91489 |
| 10 | 5    | 50  | 3    | 0.25 | 0.88 | 0.84314 | 0.91489 | 0.87755 |
| 11 | 5    | 50  | 3    | 0.1  | 0.91 | 0.89583 | 0.91489 | 0.90526 |
| 12 | 5    | 50  | 3    | 0.05 | 0.90 | 0.87755 | 0.91489 | 0.89583 |

From the above example, #1 is the result with default parameters. Starting from #2, the `max_depth` is set to 3 and `max_features` is set to 1, both values are extremely small, because the maximum depth is very low the decision tree is have little leaves, this may result an underfitting where the function becomes too general to identify data, also there are 7 features in this dataset but I limit the maximum features to consider to only 1, the algorithm might not have enough features to determine the function thus the matrices are extremely low. As we doubles the `max_depth` and `max_features` from #3 we can clearly see that the results are getting better and close to the default case, however, on #4 and #5, where the maximum depth are uncapped and maximum features are set to 4 and Max, the results is slightly lower, this doesn't make sense because more depth and features means better results, but this happend so one possible explanation is Overfitting.

Because overfitting happended, starting from #6 I picked the middle values for `max_depth` and `max_features` which yields the best result. Start `min_samples_split` from 100(there're 400 training examples so this is big), there's a significant drop in terms of the previous results, and the results are coming back as we decrease the value in #7, #8, #9. Previously I suspect #5 is overfitted, thus in #9 I used 50 as the sample split, and the result is much better than #5.

Starting from #10, `min_impurity_split` is used. `min_impurity_split` is set to 0.25 at first, but due to the threshold is too big, the tree is too little to predict the data set accuratly, thus I decrease the value down to 0.1 on #11, the result getting slightly better so I continued decreasing the value down to 0.5, which result a little bit of overfitting.

In [ ]:
import graphviz
# Graph Printing
dot_data = tree.export_graphviz(clf, filled=True, rounded=True, special_characters=True) 
graph = graphviz.Source(dot_data)
graph.render("graph")

'graph.pdf'

## **Task 2 (30 points)**:

From the Bagging and Boosting methods pick any one algorithm from each category. Implement both the algorithms using the same data. Use k-fold cross validation to find the effectiveness of both the models. Comment on the difference/similarity of the results.

In [ ]:
# evaluate bagging algorithm for classification
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_validate, KFold, StratifiedKFold
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier

# define the model
bagging = BaggingClassifier()
adaboost = AdaBoostClassifier()

# define cross-validation
kfcv = StratifiedKFold(n_splits = 10)

scoring = {'acc': 'accuracy', 'prec': 'precision', 'rec': 'recall'}
# evaluate the model & report performance
cv_bagging = cross_validate(bagging, x, y, cv = kfcv, scoring=scoring, return_train_score=True)
print('Bagging:')
print('\tAccuracy: %.3f (%.3f)' % (mean(cv_bagging['test_acc']), std(cv_bagging['test_acc'])))
print('\tPrecision: %.3f (%.3f)' % (mean(cv_bagging['test_prec']), std(cv_bagging['test_prec'])))
print('\tRecall: %.3f (%.3f)' % (mean(cv_bagging['test_rec']), std(cv_bagging['test_rec'])))

cv_boosting = cross_validate(adaboost, x, y, cv = kfcv, scoring=scoring, return_train_score=True)
print('Boosting:')
print('\tAccuracy: %.3f (%.3f)' % (mean(cv_boosting['test_acc']), std(cv_boosting['test_acc'])))
print('\tPrecision: %.3f (%.3f)' % (mean(cv_boosting['test_prec']), std(cv_boosting['test_prec'])))
print('\tRecall: %.3f (%.3f)' % (mean(cv_boosting['test_rec']), std(cv_boosting['test_rec'])))

Bagging:
	Accuracy: 0.822 (0.050)
	Precision: 0.844 (0.066)
	Recall: 0.822 (0.073)
Boosting:
	Accuracy: 0.842 (0.054)
	Precision: 0.867 (0.064)
	Recall: 0.837 (0.086)


- Record Average of `accuracy`, `precision` & `recall` scores for both Bagging and AdaBoost
  - Bagging:

|               | 1          | 2          | 3          | 4          | 5          | 6          | 7          | 8          | 9          | 10         |
| ------------- | ---------- | ---------- | ---------- | ---------- | ---------- | ---------- | ---------- | ---------- | ---------- | ---------- |
| Accuracy(std) | .822(.053) | .830(.055) | .826(.060) | .828(.053) | .818(.072) | .824(.052) | .842(.048) | .828(.052) | .826(.057) | .822(.050) |
| Precision     | .858(.081) | .868(.082) | .865(.085) | .865(.079) | .857(.087) | .858(.079) | .876(.076) | .859(.076) | .865(.080) | .844(.066) |
| Recall        | .810(.085) | .814(.087) | .810(.087) | .814(.091) | .799(.092) | .814(.089) | .829(.067) | .822(.078) | .811(.088) | .822(.073) |

  - Boosting:

|               | 1          | 2          | 3          | 4          | 5          | 6          | 7          | 8          | 9          | 10         |
| ------------- | ---------- | ---------- | ---------- | ---------- | ---------- | ---------- | ---------- | ---------- | ---------- | ---------- |
| Accuracy(std) | .842(.054) | .844(.050) | .844(.050) | .842(.054) | .844(.050) | .844(.050) | .844(.050) | .844(.050) | .842(.054) | .842(.054) |
| Precision     | .867(.064) | .869(.058) | .869(.058) | .867(.064) | .869(.058) | .869(.058) | .869(.058) | .869(.058) | .867(.064) | .867(.064) |
| Recall        | .837(.086) | .837(.086) | .837(.086) | .837(.086) | .837(.086) | .837(.086) | .837(.086) | .837(.086) | .837(.086) | .837(.086) |

These 2 tables record the mean & std of the accuracy, precision and recall of both model. From the Boosting table we can see that all scores are very consistant, the number doesn't fluctuate very much, where the bagging scores are more fluctuated and they're also lower than Boosting scores for most of the time, and the standard deviation is typically higher means that distribution of scores in bagging is more discrete.

## **Task 3 (40 points)**:

Compare the effectiveness of the three models implemented above. Clearly describe the metric you are using for comparison. Describe (with examples) Why is this metric(metrics) suited/appropriate for the problem at hand? How would a choice of a different metric impact your results? Can you demonstrate that?

#
Precision is to minimize the false positive, and recall is to minimize the false negative, in our scenario, where we would like to predict whether a student will get admitted for a college, in this case recall is better than precision because a qualified student gets refused by admission is worse than a unqualified student gets accepted, so we ruled out precision. Accuracy is good for data labels that are evenly balenced, which is the same for this dataset, but the goal is to minimize the false negative, I prefer recall over accuracy.

Among all 3 models, decision tree classifier is the best model if it have setup the correct parameters, even for some **bad** cases in the decision tree model result table the recall is still higher than bagging but just slightly lower than AdaBoost. For the best case, the recall can go as high as 0.93 which beat the other 2 models, bagging have the worst recall score and AdaBoost have average scores, thus I believe the deciision tree classifier is the best model.

If we look at precision instead, we will find out that all 3 models have roughly the same scores. In decision tree model, most of the precision score fluctuate between 0.80\~0.90. In bagging model, they are around 0.85. And in AdaBoost model, precision scores are around 0.86\~0.87, comapare with bagging, they are slightly higher than 0.01, and some of the scores in decision tree is higher than AdaBoost probably due to the correct parameter setup, but even with the best parameters, the scores in decision trees are still fluctuating pretty heavily so they may sometimes fall under AdaBoost. In conclusion, I'd say AdaBoost is the best model for precision due to how stable it is than decision trees.